# 86. 受容野入門：「視野」という概念

**Unit 0.4: 空間の帰納バイアス - CNN & 局所性の科学**

---

## このノートブックの位置づけ

Section Bでは、CNNの**受容野（Receptive Field）**を深く理解します。受容野とは、出力の1ピクセルが入力画像の「どの範囲」を見ているかを表す概念です。

```
Unit 0.4 学習マップ
==================

Section A: 畳み込みの世界への入口 ✅ 完了

Section B: 受容野と階層的抽象化
├── 86. 受容野入門：「視野」という概念  ← 今ここ
├── 87. 層を重ねる：深さが生む抽象化
├── 88. ダウンサンプリング：効率的に視野を広げる
└── 89. 受容野と3DGS：空間的影響範囲の対比
```

## 学習目標

1. **受容野の概念を直感的に理解する**
2. **人間の視覚系との類似性を認識する**
3. **1層の畳み込みの受容野を計算できる**
4. **受容野の可視化ツールを作成できる**
5. **「どこから情報が来たか」を逆伝播で追跡できる**

## 目次

1. [核となる問い：1ピクセルは世界のどこを見ているか](#1-核となる問い1ピクセルは世界のどこを見ているか)
2. [受容野の直感的理解](#2-受容野の直感的理解)
3. [1層の畳み込みの受容野](#3-1層の畳み込みの受容野)
4. [受容野の可視化ツールを作る](#4-受容野の可視化ツールを作る)
5. [実験：特定の出力ピクセルの「親」を探す](#5-実験特定の出力ピクセルの親を探す)

---

## 環境セットアップ

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle
from matplotlib.collections import PatchCollection
import warnings
warnings.filterwarnings('ignore')

try:
    import torch
    import torch.nn as nn
    TORCH_AVAILABLE = True
except ImportError:
    TORCH_AVAILABLE = False

plt.rcParams['font.family'] = ['Hiragino Sans', 'Arial Unicode MS', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

np.random.seed(42)
print("環境準備完了")

---

## 1. 核となる問い：1ピクセルは世界のどこを見ているか

CNNの出力層にある1つのピクセル（または1つのニューロン）を選んだとき、そのピクセルの値は入力画像のどの部分から計算されたのでしょうか？

これが**受容野（Receptive Field）**の核心的な問いです。

In [ ]:
# 核となる問いの図解

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# 入力画像
ax1 = axes[0]
input_size = 16
input_img = np.random.rand(input_size, input_size)
ax1.imshow(input_img, cmap='gray')
ax1.set_title('入力画像 (16×16)', fontsize=12)
ax1.set_xlabel('入力の各ピクセルは\n画像の1点の情報', fontsize=10)

# 中間：CNNの処理
ax2 = axes[1]
ax2.axis('off')
ax2.text(0.5, 0.7, 'CNN\n(複数の畳み込み層)', fontsize=14, ha='center', va='center',
        bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8))
ax2.annotate('', xy=(0.8, 0.5), xytext=(0.2, 0.5),
            arrowprops=dict(arrowstyle='->', color='black', lw=2))
ax2.set_xlim(0, 1)
ax2.set_ylim(0, 1)

# 出力
ax3 = axes[2]
output_size = 4
output_img = np.random.rand(output_size, output_size)
ax3.imshow(output_img, cmap='viridis')

# 1つのピクセルをハイライト
highlight = Rectangle((1.5, 1.5), 1, 1, fill=False, edgecolor='red', linewidth=3)
ax3.add_patch(highlight)
ax3.set_title('出力特徴マップ (4×4)', fontsize=12)
ax3.set_xlabel('この1ピクセルは\n入力のどこを見ている？', fontsize=10, color='red')

plt.suptitle('核となる問い：出力ピクセルの「視野」とは？', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n受容野（Receptive Field）とは：")
print("  出力の1ピクセルの値を計算するために参照される、入力画像の領域のこと")
print("\nなぜ重要？")
print("  - 小さい受容野 → 細かい特徴（エッジ、テクスチャ）を検出")
print("  - 大きい受容野 → 大域的な特徴（物体全体、構図）を理解")

---

## 2. 受容野の直感的理解

### 2.1 人間の網膜と受容野

受容野という概念は、神経科学から来ています。網膜の視細胞や視覚野のニューロンは、視野の特定の領域に対してのみ反応します。

In [ ]:
# 視覚系の受容野の図解

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# 視野全体
ax1 = axes[0]
ax1.set_xlim(-10, 10)
ax1.set_ylim(-10, 10)
ax1.set_aspect('equal')

# 視野の円
visual_field = Circle((0, 0), 9, fill=True, facecolor='lightblue', edgecolor='blue', alpha=0.3)
ax1.add_patch(visual_field)

# 中心視野（高解像度）
fovea = Circle((0, 0), 2, fill=True, facecolor='yellow', edgecolor='orange', alpha=0.5)
ax1.add_patch(fovea)

ax1.set_title('人間の視野', fontsize=12)
ax1.text(0, 0, '中心視野\n(高解像度)', ha='center', va='center', fontsize=9)
ax1.text(5, 5, '周辺視野\n(低解像度)', ha='center', va='center', fontsize=9)
ax1.axis('off')

# 網膜の受容野
ax2 = axes[1]
ax2.set_xlim(-5, 5)
ax2.set_ylim(-5, 5)
ax2.set_aspect('equal')

# ON中心-OFF周辺型の受容野
on_center = Circle((0, 0), 1, fill=True, facecolor='red', alpha=0.5)
off_surround = Circle((0, 0), 2.5, fill=True, facecolor='blue', alpha=0.3)
ax2.add_patch(off_surround)
ax2.add_patch(on_center)

ax2.set_title('網膜神経節細胞の受容野\n（ON中心-OFF周辺型）', fontsize=12)
ax2.text(0, 0, 'ON\n(興奮)', ha='center', va='center', fontsize=9, color='white')
ax2.text(1.8, 1.8, 'OFF\n(抑制)', ha='center', va='center', fontsize=9)
ax2.axis('off')

# CNNの受容野
ax3 = axes[2]
ax3.set_xlim(-5, 5)
ax3.set_ylim(-5, 5)
ax3.set_aspect('equal')

# 矩形の受容野（CNN）
cnn_rf = Rectangle((-1.5, -1.5), 3, 3, fill=True, facecolor='green', alpha=0.3, edgecolor='green', linewidth=2)
ax3.add_patch(cnn_rf)

# グリッド
for i in range(-1, 2):
    for j in range(-1, 2):
        rect = Rectangle((i-0.4, j-0.4), 0.8, 0.8, fill=True, 
                         facecolor='lightgreen', edgecolor='darkgreen', alpha=0.7)
        ax3.add_patch(rect)

ax3.set_title('CNNの受容野\n（3×3カーネル）', fontsize=12)
ax3.text(0, -3, '離散的なグリッド\n各セルに重み', ha='center', va='center', fontsize=9)
ax3.axis('off')

plt.suptitle('生物学的受容野 vs CNNの受容野', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n類似点：")
print("  - どちらも入力空間の限定された領域に反応")
print("  - 中心と周辺で異なる重み付け（CNNは学習で獲得）")
print("\n相違点：")
print("  - 生物：連続的、ON/OFF領域")
print("  - CNN：離散的、学習された任意の重み")

### 2.2 CNNにおける「入力への影響範囲」

In [ ]:
# CNNの受容野：1層の場合

def visualize_single_layer_rf(input_size, kernel_size):
    """
    1層の畳み込みにおける受容野を可視化
    """
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # 入力
    ax1 = axes[0]
    for i in range(input_size):
        for j in range(input_size):
            rect = Rectangle((j, input_size-1-i), 1, 1, 
                            fill=True, facecolor='lightblue', edgecolor='gray', alpha=0.5)
            ax1.add_patch(rect)
    ax1.set_xlim(-0.5, input_size+0.5)
    ax1.set_ylim(-0.5, input_size+0.5)
    ax1.set_aspect('equal')
    ax1.set_title(f'入力 ({input_size}×{input_size})', fontsize=12)
    ax1.axis('off')
    
    # カーネル
    ax2 = axes[1]
    for i in range(kernel_size):
        for j in range(kernel_size):
            rect = Rectangle((j, kernel_size-1-i), 1, 1,
                            fill=True, facecolor='orange', edgecolor='red', alpha=0.7)
            ax2.add_patch(rect)
            ax2.text(j+0.5, kernel_size-0.5-i, f'w{i*kernel_size+j}', ha='center', va='center', fontsize=9)
    ax2.set_xlim(-0.5, kernel_size+0.5)
    ax2.set_ylim(-0.5, kernel_size+0.5)
    ax2.set_aspect('equal')
    ax2.set_title(f'カーネル ({kernel_size}×{kernel_size})', fontsize=12)
    ax2.axis('off')
    
    # 出力と受容野
    ax3 = axes[2]
    output_size = input_size - kernel_size + 1
    
    # 入力を薄く描画
    for i in range(input_size):
        for j in range(input_size):
            rect = Rectangle((j, input_size-1-i), 1, 1,
                            fill=True, facecolor='lightgray', edgecolor='gray', alpha=0.3)
            ax3.add_patch(rect)
    
    # 特定の出力位置を選択
    out_i, out_j = output_size // 2, output_size // 2
    
    # その出力位置の受容野をハイライト
    for i in range(kernel_size):
        for j in range(kernel_size):
            rect = Rectangle((out_j + j, input_size-1-(out_i + i)), 1, 1,
                            fill=True, facecolor='yellow', edgecolor='red', linewidth=2, alpha=0.7)
            ax3.add_patch(rect)
    
    # 出力位置を示す矢印
    ax3.annotate(f'出力({out_i},{out_j})の\n受容野', 
                xy=(out_j + kernel_size/2, input_size - out_i - kernel_size/2),
                xytext=(input_size + 1, input_size/2),
                arrowprops=dict(arrowstyle='->', color='red', lw=2),
                fontsize=10, color='red')
    
    ax3.set_xlim(-0.5, input_size + 3)
    ax3.set_ylim(-0.5, input_size + 0.5)
    ax3.set_aspect('equal')
    ax3.set_title(f'受容野 = {kernel_size}×{kernel_size}', fontsize=12)
    ax3.axis('off')
    
    plt.suptitle(f'1層の畳み込み：受容野 = カーネルサイズ', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    return kernel_size

# 3x3カーネルの場合
rf = visualize_single_layer_rf(input_size=8, kernel_size=3)
print(f"\n結論: 1層の畳み込みでは、受容野 = カーネルサイズ = {rf}×{rf}")

---

## 3. 1層の畳み込みの受容野

1層の畳み込みでは、受容野は単純にカーネルサイズと同じです。

In [ ]:
# 異なるカーネルサイズでの受容野

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

kernel_sizes = [1, 3, 5, 7]
input_size = 11

for ax, k_size in zip(axes, kernel_sizes):
    # 入力グリッド
    for i in range(input_size):
        for j in range(input_size):
            color = 'lightgray'
            alpha = 0.3
            
            # 中心位置の受容野をハイライト
            center = input_size // 2
            half_k = k_size // 2
            if (center - half_k <= i <= center + half_k and 
                center - half_k <= j <= center + half_k):
                color = 'orange'
                alpha = 0.7
            
            rect = Rectangle((j, input_size-1-i), 1, 1,
                            fill=True, facecolor=color, edgecolor='gray', alpha=alpha)
            ax.add_patch(rect)
    
    # 中心点（出力位置）
    ax.plot(center + 0.5, center + 0.5, 'r*', markersize=15)
    
    ax.set_xlim(-0.5, input_size + 0.5)
    ax.set_ylim(-0.5, input_size + 0.5)
    ax.set_aspect('equal')
    ax.set_title(f'{k_size}×{k_size} カーネル\n受容野 = {k_size}×{k_size}', fontsize=11)
    ax.axis('off')

plt.suptitle('カーネルサイズと受容野の関係（1層の場合）', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n1層の畳み込みにおける受容野:")
print("  受容野サイズ = カーネルサイズ")
print("\n大きいカーネルの利点と欠点:")
print("  利点: 一度に広い範囲を見られる")
print("  欠点: パラメータ数が増加（k×k個の重み）")

In [ ]:
# パラメータ数の比較

kernel_sizes = [1, 3, 5, 7, 9, 11]
params = [k**2 for k in kernel_sizes]

fig, ax = plt.subplots(figsize=(10, 5))

bars = ax.bar(kernel_sizes, params, color='steelblue', edgecolor='navy')

# 値をバーの上に表示
for bar, p in zip(bars, params):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
           f'{p}', ha='center', va='bottom', fontsize=11)

ax.set_xlabel('カーネルサイズ', fontsize=12)
ax.set_ylabel('パラメータ数', fontsize=12)
ax.set_title('カーネルサイズとパラメータ数の関係', fontsize=14)
ax.set_xticks(kernel_sizes)
ax.set_xticklabels([f'{k}×{k}' for k in kernel_sizes])
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n問い: 大きな受容野を効率的に実現する方法は？")
print("答え: 小さなカーネルを複数層重ねる（次のノートブックで詳しく）")

---

## 4. 受容野の可視化ツールを作る

任意の層構成に対して受容野を計算・可視化するツールを作成します。

In [ ]:
class ReceptiveFieldCalculator:
    """
    受容野を計算するためのユーティリティクラス
    """
    
    def __init__(self):
        self.layers = []
        
    def add_conv(self, kernel_size, stride=1, padding=0, name=None):
        """
        畳み込み層を追加
        """
        self.layers.append({
            'type': 'conv',
            'kernel_size': kernel_size,
            'stride': stride,
            'padding': padding,
            'name': name or f'Conv(k={kernel_size}, s={stride})'
        })
        return self
    
    def add_pool(self, kernel_size, stride=None, name=None):
        """
        プーリング層を追加
        """
        if stride is None:
            stride = kernel_size
        self.layers.append({
            'type': 'pool',
            'kernel_size': kernel_size,
            'stride': stride,
            'padding': 0,
            'name': name or f'Pool(k={kernel_size}, s={stride})'
        })
        return self
    
    def calculate_rf(self):
        """
        受容野サイズを計算
        
        漸化式:
        RF_n = RF_{n-1} + (k_n - 1) * Π_{i=1}^{n-1} s_i
        
        または逐次計算:
        jump_n = jump_{n-1} * s_n
        rf_n = rf_{n-1} + (k_n - 1) * jump_{n-1}
        """
        rf = 1  # 初期受容野（入力の1ピクセル）
        jump = 1  # 累積ストライド
        
        rf_history = [{'layer': 'Input', 'rf': rf, 'jump': jump}]
        
        for layer in self.layers:
            k = layer['kernel_size']
            s = layer['stride']
            
            # 受容野の更新
            rf = rf + (k - 1) * jump
            
            rf_history.append({
                'layer': layer['name'],
                'rf': rf,
                'jump': jump * s
            })
            
            # ジャンプ（累積ストライド）の更新
            jump = jump * s
        
        return rf, rf_history
    
    def visualize(self):
        """
        受容野の成長を可視化
        """
        rf, history = self.calculate_rf()
        
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # 左：受容野の成長グラフ
        ax1 = axes[0]
        layers_names = [h['layer'] for h in history]
        rf_values = [h['rf'] for h in history]
        
        ax1.bar(range(len(rf_values)), rf_values, color='steelblue', edgecolor='navy')
        ax1.set_xticks(range(len(rf_values)))
        ax1.set_xticklabels(layers_names, rotation=45, ha='right', fontsize=9)
        ax1.set_ylabel('受容野サイズ', fontsize=11)
        ax1.set_title('層ごとの受容野の成長', fontsize=12)
        ax1.grid(True, alpha=0.3, axis='y')
        
        # 値をバーの上に表示
        for i, v in enumerate(rf_values):
            ax1.text(i, v + 0.5, f'{v}', ha='center', fontsize=10)
        
        # 右：受容野の視覚化
        ax2 = axes[1]
        max_rf = max(rf_values)
        colors = plt.cm.viridis(np.linspace(0, 1, len(rf_values)))
        
        for i, (r, color) in enumerate(zip(rf_values, colors)):
            rect = Rectangle((-r/2, -r/2), r, r, fill=False, 
                            edgecolor=color, linewidth=2, alpha=0.8)
            ax2.add_patch(rect)
        
        ax2.set_xlim(-max_rf/2 - 2, max_rf/2 + 2)
        ax2.set_ylim(-max_rf/2 - 2, max_rf/2 + 2)
        ax2.set_aspect('equal')
        ax2.set_title('受容野の拡大（同心正方形）', fontsize=12)
        ax2.grid(True, alpha=0.3)
        
        plt.suptitle(f'最終受容野: {rf}×{rf} ピクセル', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()
        
        return rf, history
    
    def print_summary(self):
        """
        受容野の計算過程をテーブル形式で表示
        """
        rf, history = self.calculate_rf()
        
        print("\n" + "="*60)
        print("受容野計算サマリー")
        print("="*60)
        print(f"{'層':<25} {'受容野':>10} {'ジャンプ':>10}")
        print("-"*60)
        
        for h in history:
            print(f"{h['layer']:<25} {h['rf']:>10} {h['jump']:>10}")
        
        print("="*60)
        print(f"最終受容野: {rf}×{rf} ピクセル")
        print("="*60)
        
        return rf

In [ ]:
# 使用例：シンプルな3層CNN

calc = ReceptiveFieldCalculator()
calc.add_conv(kernel_size=3, stride=1, name='Conv1 (3×3)')
calc.add_conv(kernel_size=3, stride=1, name='Conv2 (3×3)')
calc.add_conv(kernel_size=3, stride=1, name='Conv3 (3×3)')

rf, history = calc.visualize()
calc.print_summary()

print("\n重要な発見:")
print("  3×3カーネルを3層重ねると、受容野は7×7になる")
print("  → 7×7カーネル1層と同じ受容野だが、パラメータ数は 3×(3×3)=27 vs 7×7=49")

In [ ]:
# 使用例：VGG風のネットワーク

calc_vgg = ReceptiveFieldCalculator()

# Block 1
calc_vgg.add_conv(3, 1, name='Conv1-1')
calc_vgg.add_conv(3, 1, name='Conv1-2')
calc_vgg.add_pool(2, 2, name='Pool1')

# Block 2
calc_vgg.add_conv(3, 1, name='Conv2-1')
calc_vgg.add_conv(3, 1, name='Conv2-2')
calc_vgg.add_pool(2, 2, name='Pool2')

# Block 3
calc_vgg.add_conv(3, 1, name='Conv3-1')
calc_vgg.add_conv(3, 1, name='Conv3-2')
calc_vgg.add_conv(3, 1, name='Conv3-3')
calc_vgg.add_pool(2, 2, name='Pool3')

rf_vgg, _ = calc_vgg.visualize()
calc_vgg.print_summary()

---

## 5. 実験：特定の出力ピクセルの「親」を探す

実際にニューラルネットワークを使って、出力から入力への「情報の流れ」を追跡します。

In [ ]:
def trace_receptive_field_manual(input_size, layers_config, output_pos):
    """
    手動で受容野を逆追跡
    
    Parameters:
    -----------
    input_size : int
        入力画像サイズ
    layers_config : list of dict
        各層の設定 {'kernel_size': k, 'stride': s, 'padding': p}
    output_pos : tuple (row, col)
        追跡したい出力位置
    """
    # 各層の出力サイズを計算
    sizes = [input_size]
    for layer in layers_config:
        k, s, p = layer['kernel_size'], layer['stride'], layer['padding']
        new_size = (sizes[-1] + 2*p - k) // s + 1
        sizes.append(new_size)
    
    # 出力から入力へ逆追跡
    # 各層での「見ている範囲」を計算
    current_range = [(output_pos[0], output_pos[0]), (output_pos[1], output_pos[1])]  # [(row_min, row_max), (col_min, col_max)]
    ranges_history = [current_range.copy()]
    
    for layer in reversed(layers_config):
        k, s, p = layer['kernel_size'], layer['stride'], layer['padding']
        
        # 前の層での範囲を計算
        row_min = current_range[0][0] * s - p
        row_max = current_range[0][1] * s + k - 1 - p
        col_min = current_range[1][0] * s - p
        col_max = current_range[1][1] * s + k - 1 - p
        
        current_range = [(row_min, row_max), (col_min, col_max)]
        ranges_history.append(current_range.copy())
    
    ranges_history.reverse()
    
    return sizes, ranges_history

# 例：3層の3x3畳み込み
input_size = 11
layers = [
    {'kernel_size': 3, 'stride': 1, 'padding': 0},
    {'kernel_size': 3, 'stride': 1, 'padding': 0},
    {'kernel_size': 3, 'stride': 1, 'padding': 0},
]

# 出力の中心位置を追跡
output_size = input_size
for layer in layers:
    output_size = output_size - layer['kernel_size'] + 1

output_pos = (output_size // 2, output_size // 2)

sizes, ranges = trace_receptive_field_manual(input_size, layers, output_pos)

print(f"入力サイズ: {input_size}×{input_size}")
print(f"出力サイズ: {output_size}×{output_size}")
print(f"\n出力位置 {output_pos} の受容野追跡:")
print("="*50)

layer_names = ['入力'] + [f'Conv{i+1}後' for i in range(len(layers))]
for name, size, range_info in zip(layer_names, sizes, ranges):
    print(f"{name}: サイズ{size}×{size}, 範囲 rows[{range_info[0][0]}:{range_info[0][1]+1}], cols[{range_info[1][0]}:{range_info[1][1]+1}]")

In [ ]:
# 受容野の逆追跡を可視化

def visualize_rf_backtrack(input_size, layers, output_pos):
    """
    受容野の逆追跡を可視化
    """
    sizes, ranges = trace_receptive_field_manual(input_size, layers, output_pos)
    
    n_layers = len(layers) + 1
    fig, axes = plt.subplots(1, n_layers, figsize=(4 * n_layers, 4))
    
    layer_names = ['入力'] + [f'Layer {i+1}' for i in range(len(layers))]
    
    for idx, (ax, size, range_info, name) in enumerate(zip(axes, sizes, ranges, layer_names)):
        # グリッドを描画
        for i in range(size):
            for j in range(size):
                # 受容野内かどうかを判定
                in_rf = (range_info[0][0] <= i <= range_info[0][1] and
                        range_info[1][0] <= j <= range_info[1][1])
                
                color = 'orange' if in_rf else 'lightgray'
                alpha = 0.8 if in_rf else 0.3
                
                rect = Rectangle((j, size-1-i), 1, 1,
                                fill=True, facecolor=color, edgecolor='gray', alpha=alpha)
                ax.add_patch(rect)
        
        # 最終層では出力位置をマーク
        if idx == n_layers - 1:
            ax.plot(output_pos[1] + 0.5, size - output_pos[0] - 0.5, 'r*', markersize=20)
        
        ax.set_xlim(-0.5, size + 0.5)
        ax.set_ylim(-0.5, size + 0.5)
        ax.set_aspect('equal')
        ax.set_title(f'{name}\n({size}×{size})', fontsize=11)
        ax.axis('off')
        
        # 矢印で接続
        if idx < n_layers - 1:
            ax.annotate('', xy=(size + 0.8, size/2), xytext=(size + 0.2, size/2),
                       arrowprops=dict(arrowstyle='->', color='black', lw=2))
    
    plt.suptitle(f'受容野の逆追跡：出力{output_pos}が見ている入力領域', 
                fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

visualize_rf_backtrack(11, layers, output_pos)

print("\nオレンジ色の領域 = その層で『見ている』範囲")
print("入力層のオレンジ = 受容野")

In [ ]:
if TORCH_AVAILABLE:
    # PyTorchを使った受容野の可視化（勾配ベース）
    
    def visualize_rf_gradient(model, input_size, output_pos):
        """
        勾配を使って受容野を可視化
        特定の出力位置からの勾配が非ゼロになる入力位置 = 受容野
        """
        # 入力画像（勾配計算用）
        x = torch.zeros(1, 1, input_size, input_size, requires_grad=True)
        
        # 順伝播
        y = model(x)
        
        # 特定の出力位置だけを選択
        out_h, out_w = y.shape[2], y.shape[3]
        target = torch.zeros_like(y)
        target[0, 0, output_pos[0], output_pos[1]] = 1.0
        
        # 逆伝播
        y.backward(target)
        
        # 勾配を取得
        grad = x.grad[0, 0].numpy()
        
        return grad, (out_h, out_w)
    
    # シンプルな3層CNN
    class SimpleCNN(nn.Module):
        def __init__(self):
            super().__init__()
            self.conv1 = nn.Conv2d(1, 1, 3, bias=False)
            self.conv2 = nn.Conv2d(1, 1, 3, bias=False)
            self.conv3 = nn.Conv2d(1, 1, 3, bias=False)
            
            # 重みを1に設定（受容野の可視化用）
            with torch.no_grad():
                self.conv1.weight.fill_(1.0)
                self.conv2.weight.fill_(1.0)
                self.conv3.weight.fill_(1.0)
        
        def forward(self, x):
            x = self.conv1(x)
            x = self.conv2(x)
            x = self.conv3(x)
            return x
    
    model = SimpleCNN()
    
    # 受容野の可視化
    input_size = 15
    grad, (out_h, out_w) = visualize_rf_gradient(model, input_size, (out_h//2, out_w//2) if 'out_h' in dir() else (2, 2))
    
    # 出力サイズを計算
    x_test = torch.zeros(1, 1, input_size, input_size)
    y_test = model(x_test)
    out_h, out_w = y_test.shape[2], y_test.shape[3]
    
    # 中心の出力位置での受容野
    output_pos = (out_h // 2, out_w // 2)
    grad, _ = visualize_rf_gradient(model, input_size, output_pos)
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # 勾配の可視化
    ax1 = axes[0]
    im = ax1.imshow(grad, cmap='hot')
    ax1.set_title(f'勾配マップ\n（出力位置 {output_pos}）', fontsize=12)
    plt.colorbar(im, ax=ax1)
    
    # 二値化した受容野
    ax2 = axes[1]
    rf_mask = (grad != 0).astype(float)
    ax2.imshow(rf_mask, cmap='Blues')
    ax2.set_title(f'受容野（勾配≠0の領域）\nサイズ: {int(rf_mask.sum()**0.5)}×{int(rf_mask.sum()**0.5)}', fontsize=12)
    
    plt.suptitle('勾配ベースの受容野可視化', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print(f"\n入力サイズ: {input_size}×{input_size}")
    print(f"出力サイズ: {out_h}×{out_w}")
    print(f"受容野サイズ: {int(rf_mask.sum()**0.5)}×{int(rf_mask.sum()**0.5)}")

---

## まとめ

In [ ]:
summary = """
╔═══════════════════════════════════════════════════════════════════════════════╗
║                    受容野入門：キーポイント                                    ║
╠═══════════════════════════════════════════════════════════════════════════════╣
║                                                                               ║
║  【受容野（Receptive Field）とは】                                             ║
║  • 出力の1ピクセルが「見ている」入力画像の領域                                ║
║  • CNNの「視野」に相当する概念                                                ║
║                                                                               ║
║  【1層の畳み込み】                                                             ║
║  • 受容野 = カーネルサイズ                                                    ║
║  • 3×3カーネル → 3×3の受容野                                                  ║
║                                                                               ║
║  【受容野の計算】                                                              ║
║  • 漸化式: RF_n = RF_{n-1} + (k_n - 1) × 累積ストライド                       ║
║  • 層を重ねるほど受容野は拡大                                                 ║
║                                                                               ║
║  【なぜ重要か】                                                                ║
║  • 小さい受容野: 局所的な特徴（エッジ、テクスチャ）                           ║
║  • 大きい受容野: 大域的な特徴（物体、構図）                                   ║
║  • タスクに応じた適切な受容野設計が重要                                       ║
║                                                                               ║
╚═══════════════════════════════════════════════════════════════════════════════╝
"""
print(summary)

---

## 次のステップ

次のノートブック **87. 層を重ねる：深さが生む抽象化** では：

- 受容野の漸化式を詳しく導出
- 3×3を2回 = 5×5相当の証明
- パラメータ効率の比較
- 有効受容野（Effective Receptive Field）の概念
- 階層的特徴抽出のイメージ

「小さなカーネルを深く重ねる」という現代CNNの設計思想の数学的根拠を学びます。